In [330]:
#-------------------------------------------------------Library imports----------------------------------------------------------
#Importo la liberia numpy perchè è un pacchetto che fornisce oggetti di tipo array multidimensionali, operazioni veloci sui array etc
import numpy as np
#Importo la libreria di panda perchè : esistono metodi build-in in python che si sovrapongono con i metodi di panda.
#Es map() / all() / any() / filter() etc. Per evitare le collisioni di questi metodi importo la libreria di panda
import pandas as pd
#Importo la libreria seaborn per poter visualizzare i drawing 
import seaborn as sns
#Importo la libreria di pickle per poter serializzare e deserializzare le 
#strutture di oggetti Python chiamate anche marshalling o appiattimento.
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn import model_selection 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import os.path as osp
import os
import logging
from glob import glob
import numpy as np
import re   
import plotnine as pln
from matplotlib import pyplot as plt


import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

#-------------------------------------------------------Dataset import----------------------------------------------------------
import_dataset = 'datasets/ATP_dataset_2019_2011.xlsx'
imd = pd.read_excel(import_dataset)

In [331]:
np.random.seed(870383)

In [332]:
imd.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,Wsets,Lsets,Weather
0,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3,Dimitrov G.,Simon G.,28.0,15.0,1360.0,2145.0,2.0,0.0,24
1,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3,Kudla D.,Smith J.P.,69.0,129.0,719.0,459.0,2.0,1.0,21
2,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3,Kamke T.,Mitchell B.,277.0,231.0,185.0,225.0,2.0,0.0,25
3,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3,Chung H.,Groth S.,51.0,60.0,817.0,782.0,2.0,0.0,20
4,1,Brisbane,Brisbane International,2016-01-05,ATP250,Outdoor,Hard,1st Round,3,Goffin D.,Bellucci T.,16.0,37.0,1880.0,1105.0,2.0,0.0,22


In [333]:
def processing(data):
    imd.drop(axis=1,columns=['Location','Tournament', 'Date'], inplace=True)
    imd['Series'] = imd['Series'].map({'ATP250':1,'ATP500':2,'Master 1000':3,'Master Cup':4,'Grand Slam':5})
    imd['Court'] = imd['Court'].map({'Outdoor':1,'Indorr':2})
    imd['Surface'] = imd['Surface'].map({'Hard':1,'Clay':2,'Grass':3})
    

In [334]:
processing(imd)

In [335]:
vit = 0
imd['Vittoria'] = vit
imd.head()

,ATP,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,Wsets,Lsets,Weather,Vittoria
0,1,1.0,1.0,1,1st Round,3,Dimitrov G.,Simon G.,28.0,15.0,1360.0,2145.0,2.0,0.0,24,0
1,1,1.0,1.0,1,1st Round,3,Kudla D.,Smith J.P.,69.0,129.0,719.0,459.0,2.0,1.0,21,0
2,1,1.0,1.0,1,1st Round,3,Kamke T.,Mitchell B.,277.0,231.0,185.0,225.0,2.0,0.0,25,0
3,1,1.0,1.0,1,1st Round,3,Chung H.,Groth S.,51.0,60.0,817.0,782.0,2.0,0.0,20,0
4,1,1.0,1.0,1,1st Round,3,Goffin D.,Bellucci T.,16.0,37.0,1880.0,1105.0,2.0,0.0,22,0


In [336]:
from random import randint
def swap_values(dataset):
    for t in range(4000):
        value = randint(0, 4000)
        dataset.loc[value,['Winner','Loser']] = dataset.loc[value,['Loser','Winner']].values
        dataset.loc[value,['WRank','LRank' ]] = dataset.loc[value,['LRank','WRank' ]].values
        dataset.loc[value,['WPts','LPts'   ]] = dataset.loc[value,['LPts','WPts'   ]].values
        dataset.loc[value,['Wsets','Lsets' ]] = dataset.loc[value,['Lsets','Wsets' ]].values
        dataset.loc[value,['Vittoria']]=1
    

In [337]:
imd.head()

,ATP,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,Wsets,Lsets,Weather,Vittoria
0,1,1.0,1.0,1,1st Round,3,Dimitrov G.,Simon G.,28.0,15.0,1360.0,2145.0,2.0,0.0,24,0
1,1,1.0,1.0,1,1st Round,3,Kudla D.,Smith J.P.,69.0,129.0,719.0,459.0,2.0,1.0,21,0
2,1,1.0,1.0,1,1st Round,3,Kamke T.,Mitchell B.,277.0,231.0,185.0,225.0,2.0,0.0,25,0
3,1,1.0,1.0,1,1st Round,3,Chung H.,Groth S.,51.0,60.0,817.0,782.0,2.0,0.0,20,0
4,1,1.0,1.0,1,1st Round,3,Goffin D.,Bellucci T.,16.0,37.0,1880.0,1105.0,2.0,0.0,22,0


In [338]:
swap_values(imd)

In [339]:
imd.head()

,ATP,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,Wsets,Lsets,Weather,Vittoria
0,1,1.0,1.0,1,1st Round,3,Simon G.,Dimitrov G.,15.0,28.0,2145.0,1360.0,0.0,2.0,24,1
1,1,1.0,1.0,1,1st Round,3,Smith J.P.,Kudla D.,129.0,69.0,459.0,719.0,1.0,2.0,21,1
2,1,1.0,1.0,1,1st Round,3,Kamke T.,Mitchell B.,277.0,231.0,185.0,225.0,2.0,0.0,25,1
3,1,1.0,1.0,1,1st Round,3,Chung H.,Groth S.,51.0,60.0,817.0,782.0,2.0,0.0,20,0
4,1,1.0,1.0,1,1st Round,3,Bellucci T.,Goffin D.,37.0,16.0,1105.0,1880.0,0.0,2.0,22,1


In [340]:
featuresCols = np.array(['Vittoria','Series','Court','Surface','WRank','LRank','WPts','LPts','Wsets','Lsets','Weather'])

In [341]:
imd=imd.dropna()
TrainData = imd.loc[:,featuresCols].copy()
TrainX = TrainData.loc[:,'Vittoria'!=featuresCols]
TrainY = TrainData.loc[:,'Vittoria']

In [342]:
TrainData.head()

,Vittoria,Series,Court,Surface,WRank,LRank,WPts,LPts,Wsets,Lsets,Weather
0,1,1.0,1.0,1,15.0,28.0,2145.0,1360.0,0.0,2.0,24
1,1,1.0,1.0,1,129.0,69.0,459.0,719.0,1.0,2.0,21
2,1,1.0,1.0,1,277.0,231.0,185.0,225.0,2.0,0.0,25
3,0,1.0,1.0,1,51.0,60.0,817.0,782.0,2.0,0.0,20
4,1,1.0,1.0,1,37.0,16.0,1105.0,1880.0,0.0,2.0,22


In [343]:
dataset = TrainData.astype(float).values
X = dataset[:,1:]
print("X shape", X.shape)

# get class label
y = dataset[:,0]

X shape (8306, 10)


In [344]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [345]:
RanFo_algo=RandomForestClassifier(n_estimators=1000, random_state=1995, min_samples_leaf=15, max_features=10, criterion='gini' )

In [346]:
RanFo_algo.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=10, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=15, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=1995,
                       verbose=0, warm_start=False)

In [347]:
rf_pred = RanFo_algo.predict(X_test)

In [348]:
RanFo_algo.score(X_test, y_test)

0.9452466907340553

In [349]:
from sklearn import neighbors
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split


for k in range(1,20):

    kNN = neighbors.KNeighborsClassifier(n_neighbors=k)
    kNN.fit(X_train,y_train)

    y_pred = kNN.predict(X_test)

    # compute Accuracy
    print ("k:", k," | Accuracy:", accuracy_score(y_true=y_test, y_pred=y_pred) )

k: 1  | Accuracy: 0.7448856799037304
k: 2  | Accuracy: 0.8194945848375451
k: 3  | Accuracy: 0.7942238267148014
k: 4  | Accuracy: 0.8285198555956679
k: 5  | Accuracy: 0.8176895306859205
k: 6  | Accuracy: 0.8279181708784596
k: 7  | Accuracy: 0.8249097472924187
k: 8  | Accuracy: 0.8249097472924187
k: 9  | Accuracy: 0.8267148014440433
k: 10  | Accuracy: 0.8273164861612515
k: 11  | Accuracy: 0.8315282791817088
k: 12  | Accuracy: 0.8315282791817088
k: 13  | Accuracy: 0.8357400722021661
k: 14  | Accuracy: 0.8357400722021661
k: 15  | Accuracy: 0.8357400722021661
k: 16  | Accuracy: 0.8363417569193743
k: 17  | Accuracy: 0.8369434416365824
k: 18  | Accuracy: 0.8363417569193743
k: 19  | Accuracy: 0.8369434416365824
